# MongoDB with PyMongo
This notebook is meant for loading the data obtained from the Siamese network to MongoDB ATLAS with Python.

Some resources:

https://docs.atlas.mongodb.com/getting-started/

https://api.mongodb.com/python/current/

In [ ]:
!pip install pymongo

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# change with the PATH where you previously stored the .json files related to each movie
%cd drive/MyDrive/MDP/Dataset

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/MDP/Dataset


You need to change your username, password and cluster to create the connection URI

In [ ]:
import pymongo
from urllib.parse import quote_plus
username = quote_plus('')
password = quote_plus('')
cluster = 'cluster0.hns6k.mongodb.net'
uri = 'mongodb+srv://' + username + ':' + password + '@' + cluster
client = pymongo.MongoClient(uri)

In [ ]:
db = client.movies

In [ ]:
db.movies_info

Creates a document representing a movie in the following format  
* title
* frames: Array whose elements are
      {
      "_id": ObjectId(12-bytes string with frame_id)
      "Coordinates": [ [x1,y1,x2,y2] ],    
      }



In [ ]:
import json
from bson.objectid import ObjectId
import pprint

file_descriptor = open('Documents/iron_man_vs_loki.json',"r")
data = json.load(file_descriptor)
document = {}

frame_ids = [int(key) for key in data.keys()]


document['title'] = "Iron Man vs Loki"
document['frames'] = [{"_id": ObjectId((id).to_bytes(12,'big')), "Coordinates": [data[str(id)][box_id]['Coordinates']]} 
                      for id in frame_ids for box_id in data[str(id)].keys()]

db.movies_info.insert_one(document)

Query to ask for the information related to the **frame_id**

In [ ]:
frame_id = 0
bytes_id = frame_id.to_bytes(12,'big')
vals = db.movies_info.aggregate([{"$match": {"title": "Iron Man vs Loki"}},
                                 {"$project":{"frame": { "$arrayElemAt": [ "$frames",0]}}}])
returned = list(vals)

print(returned[0]["frame"]["Coordinates"])